In [1]:
!pip install -qq adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.0/256.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 8.6 MB/s eta 0:00:00


In [25]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 474.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/AdaptOrDie'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/AdaptOrDie


In [2]:
import json
import os
from IPython.display import Image

In [3]:
def load_json(path):
  with open(path) as f:
      data = [json.loads(line) for line in f]
  return data

def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# **RoBERTa Adapter-Base Model**

### **Loading model and adapter framework**

In [4]:
from transformers import RobertaTokenizer
from transformers import RobertaConfig
from adapters import AutoAdapterModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Pretrained RoBERTa model from Hugging Face
config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Add a new adapter to model
model.add_adapter("imbd_adapter", config="seq_bn")

# Add a matching classification head
model.add_classification_head(
    "imbd_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("imbd_adapter")

In [6]:
# Add a new adapter to model
model.add_adapter("helpfulness_adapter", config="seq_bn")

# Add a matching classification head
model.add_classification_head(
    "helpfulness_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("helpfulness_adapter")

# **Gurungaran DataSet**

## **Classification Review Datasets**

In [7]:
ds= {"imdb": {
        "data_dir": "https://s3-us-west-2.amazonaws.com/allennlp/dont_stop_pretraining/data/imdb/",
        "dataset_size": 20000
    },
    "amazon": {
        "data_dir": "https://s3-us-west-2.amazonaws.com/allennlp/dont_stop_pretraining/data/amazon/",
        "dataset_size": 115251
    }}

create_dir('datasets')
create_dir('datasets/classifier')
create_dir('datasets/classifier/imdb')
create_dir('datasets/classifier/amazon')

### **IMDB Dataset**

In [8]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/train.jsonl --output-dir 'datasets/classifier/imdb'
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/dev.jsonl --output-dir 'datasets/classifier/imdb'
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/test.jsonl --output-dir 'datasets/classifier/imdb'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.6M  100 25.6M    0     0  25.8M      0 --:--:-- --:--:-- --:--:-- 25.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6651k  100 6651k    0     0  9290k      0 --:--:-- --:--:-- --:--:-- 9289k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.4M  100 31.4M    0     0  27.3M      0  0:00:01  0:00:01 --:--:-- 27.3M


In [9]:
train_json = load_json("datasets/classifier/imdb/train.jsonl")
dev_json = load_json("datasets/classifier/imdb/dev.jsonl")
test_json = load_json("datasets/classifier/imdb/test.jsonl")
print("IMDB ds stat")
print("Train ds: {0}".format(len(train_json)))
print("Dev ds: {0}".format(len(dev_json)))
print("Test ds: {0}".format(len(test_json)))

IMDB ds stat
Train ds: 20000
Dev ds: 5000
Test ds: 25000


### **Helfulness Dataset**

In [10]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/train.jsonl --output-dir 'datasets/classifier/amazon'
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/dev.jsonl --output-dir 'datasets/classifier/amazon'
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/test.jsonl --output-dir 'datasets/classifier/amazon'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100M  100  100M    0     0  29.4M      0  0:00:03  0:00:03 --:--:-- 29.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4523k  100 4523k    0     0  6829k      0 --:--:-- --:--:-- --:--:-- 6823k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  24.0M      0 --:--:-- --:--:-- --:--:-- 24.0M


In [11]:
train_json_h = load_json("datasets/classifier/amazon/train.jsonl")
dev_json_h = load_json("datasets/classifier/amazon/dev.jsonl")
test_json_h = load_json("datasets/classifier/amazon/test.jsonl")
print("Helfulness ds stat")
print("Train ds: {0}".format(len(train_json_h)))
print("Dev ds: {0}".format(len(dev_json_h)))
print("Test ds: {0}".format(len(test_json_h)))

Helfulness ds stat
Train ds: 115251
Dev ds: 5000
Test ds: 25000


### **Combining Datasets**

In [12]:
# Combining training datasets
json_files = ["datasets/classifier/imdb/train.jsonl", "datasets/classifier/imdb/dev.jsonl", "datasets/classifier/imdb/test.jsonl", "datasets/classifier/amazon/train.jsonl", "datasets/classifier/amazon/dev.jsonl", "datasets/classifier/amazon/test.jsonl"]
reviews_datasets = []

# Load each JSON file into a Python object.
for json_file in json_files:
  a = load_json(json_file)
  reviews_datasets.extend(a)

# Dump all the Python objects into a single JSON file.
with open("combined.json", "w") as f:
    json.dump(reviews_datasets, f, indent=4)

# **Training**

Continued pretraining of RoBERTa on two
categories of unlabeled data: (i) large corpora of
domain-specific text, and (ii) available unlabeled data associated with a given task


In [13]:
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [14]:
# Encode dataset - need to tokenize json data
# Encoded dataset for transformer model - user Roberta Tokenizer
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

In [15]:
from datasets import Dataset
review_data = Dataset.from_list(reviews_datasets)

ArrowInvalid: Could not convert 'helpful' with type str: tried to convert to int64

In [17]:
# imdb
imdb_data = [train_json, dev_json, test_json]

imdb_combined = [x for xs in imdb_data for x in xs]

In [18]:
imdb_data = Dataset.from_list(imdb_combined)

In [19]:
# Encode the input data
imdb_data = imdb_data.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = imdb_data.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## **Domain-Adaptive**

In [20]:
# CLEAN MODEL
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Pretrained RoBERTa model from Hugging Face
config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
clean_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model has added adapters

### **Reviews Dataset**

In [21]:
# Train model
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=clean_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## **Task-Adaptive**

### **Reviews Dataset**

In [ ]:
# Train model

# **Evaluation**

# **Adapter Training**

# **Adapter Evaluation**